In [6]:
import numpy as np
import pandas as pd
import os

def get_run(seed,experiment,root):
    log = pd.read_csv(os.path.join(root,str(seed),experiment,"log.csv"))
    test = log.loc[log["mode"]=="test"]
    return test.iloc[-1]#.sort_values(by="kappa", ascending=False).iloc[0]

def table_row(means, stds,sigmainterval=1):
    line = list()
    for mean, std in zip(means, stds):
        line.append(f"{mean:.2f}" + "$^{\pm" + f"{std*sigmainterval:.2f}" + "}$")
    return " & ".join(line)

def get_acc_kappa(root):

    isprs_tum_rnn = [get_run(seed,"isprs_tum_rnn",root) for seed in [0,1,2]]
    isprs_gaf_rnn = [get_run(seed,"isprs_gaf_rnn",root) for seed in [0,1,2]]
    isprs_tum_transformer = [get_run(seed,"isprs_tum_transformer",root) for seed in [0,1,2]]
    isprs_gaf_transformer = [get_run(seed,"isprs_gaf_transformer",root) for seed in [0,1,2]]
    isprs_tum_msresnet = [get_run(seed,"isprs_tum_msresnet",root) for seed in [0,1,2]]
    isprs_gaf_msresnet = [get_run(seed,"isprs_gaf_msresnet",root) for seed in [0,1,2]]
    isprs_tum_tempcnn = [get_run(seed,"isprs_tum_tempcnn",root) for seed in [0,1,2]]
    isprs_gaf_tempcnn = [get_run(seed,"isprs_gaf_tempcnn",root) for seed in [0,1,2]]

    isprs_tum_rnn_kappa = np.array([run.kappa for run in isprs_tum_rnn])
    isprs_gaf_rnn_kappa = np.array([run.kappa for run in isprs_gaf_rnn])
    isprs_tum_transformer_kappa = np.array([run.kappa for run in isprs_tum_transformer])
    isprs_gaf_transformer_kappa = np.array([run.kappa for run in isprs_gaf_transformer])
    isprs_tum_msresnet_kappa = np.array([run.kappa for run in isprs_tum_msresnet])
    isprs_gaf_msresnet_kappa = np.array([run.kappa for run in isprs_gaf_msresnet])
    isprs_tum_tempcnn_kappa = np.array([run.kappa for run in isprs_tum_tempcnn])
    isprs_gaf_tempcnn_kappa = np.array([run.kappa for run in isprs_gaf_tempcnn])

    isprs_tum_rnn_accuracy = np.array([run.accuracy for run in isprs_tum_rnn])
    isprs_gaf_rnn_accuracy = np.array([run.accuracy for run in isprs_gaf_rnn])
    isprs_tum_transformer_accuracy = np.array([run.accuracy for run in isprs_tum_transformer])
    isprs_gaf_transformer_accuracy = np.array([run.accuracy for run in isprs_gaf_transformer])
    isprs_tum_msresnet_accuracy = np.array([run.accuracy for run in isprs_tum_msresnet])
    isprs_gaf_msresnet_accuracy = np.array([run.accuracy for run in isprs_gaf_msresnet])
    isprs_tum_tempcnn_accuracy = np.array([run.accuracy for run in isprs_tum_tempcnn])
    isprs_gaf_tempcnn_accuracy = np.array([run.accuracy for run in isprs_gaf_tempcnn])

    tum_kappa = [isprs_tum_rnn_kappa.mean(),isprs_tum_transformer_kappa.mean(),isprs_tum_msresnet_kappa.mean(),isprs_tum_tempcnn_kappa.mean()]
    tum_kappa_std = [isprs_tum_rnn_kappa.std(),isprs_tum_transformer_kappa.std(),isprs_tum_msresnet_kappa.std(),isprs_tum_tempcnn_kappa.std()]

    gaf_kappa = [isprs_gaf_rnn_kappa.mean(),isprs_gaf_transformer_kappa.mean(),isprs_gaf_msresnet_kappa.mean(),isprs_gaf_tempcnn_kappa.mean()]
    gaf_kappa_std = [isprs_gaf_rnn_kappa.std(),isprs_gaf_transformer_kappa.std(),isprs_gaf_msresnet_kappa.std(),isprs_gaf_tempcnn_kappa.std()]

    tum_acc = [isprs_tum_rnn_accuracy.mean(),isprs_tum_transformer_accuracy.mean(),isprs_tum_msresnet_accuracy.mean(),isprs_tum_tempcnn_accuracy.mean()]
    tum_acc_std = [isprs_tum_rnn_accuracy.std(),isprs_tum_transformer_accuracy.std(),isprs_tum_msresnet_accuracy.std(),isprs_tum_tempcnn_accuracy.std()]

    gaf_acc = [isprs_gaf_rnn_accuracy.mean(),isprs_gaf_transformer_accuracy.mean(),isprs_gaf_msresnet_accuracy.mean(),isprs_gaf_tempcnn_accuracy.mean()]
    gaf_acc_std = [isprs_gaf_rnn_accuracy.std(),isprs_gaf_transformer_accuracy.std(),isprs_gaf_msresnet_accuracy.std(),isprs_gaf_tempcnn_accuracy.std()]

    return tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std
    
def fill_table(rows,desc="ov.acc"):
    return r"""\begin{tabular}{lccccc}
\toprule
"""+desc+r""" & Random Forest & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}
    """+rows+r"""
\bottomrule
\end{tabular}"""

def make_table(exp1,exp1_std,exp2,exp2_std, desc, rf_pre, rf_raw):
    
    line1 = f"\tprocessed & {rf_pre:.2f} & "+table_row(exp1,exp1_std,sigmainterval=1) + " \\\ "
    line2 = f"\traw       & {rf_raw:.2f} & "+table_row(exp2,exp2_std,sigmainterval=1) + " \\\ "
    rows = "\n".join([line1,line2])
    
    return fill_table(rows,desc)

In [7]:
tableroot = "/tmp/"#"/home/marc/projects/ISPRS_CropTypeMapping/tables/preraw"
root = "../models/train/23classes"

tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std = get_acc_kappa(root)

# see tslearn.ipynb
rf_preprocessed_kappa = 0.58
rf_raw_kappa = 0.28
rf_preprocessed_acc = 0.64
rf_raw_acc = 0.33


print("kappa")
print()
table=make_table(gaf_kappa,gaf_kappa_std,tum_kappa,tum_kappa_std, "kappa",rf_pre=rf_preprocessed_kappa, rf_raw=rf_raw_kappa)
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("accuracy")
print()
table=make_table(gaf_acc,gaf_acc_std,tum_acc,tum_acc_std, "accuracy",rf_pre=rf_preprocessed_acc, rf_raw=rf_raw_acc)
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lccccc}
\toprule
kappa & Random Forest & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}
    	processed & 0.58 & 0.78$^{\pm0.01}$ & 0.79$^{\pm0.02}$ & 0.76$^{\pm0.02}$ & 0.80$^{\pm0.00}$ \\ 
	raw       & 0.28 & 0.71$^{\pm0.01}$ & 0.71$^{\pm0.02}$ & 0.69$^{\pm0.02}$ & 0.67$^{\pm0.01}$ \\ 
\bottomrule
\end{tabular}

writing to /tmp/kappa.tex

accuracy

\begin{tabular}{lccccc}
\toprule
accuracy & Random Forest & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}
    	processed & 0.64 & 0.85$^{\pm0.01}$ & 0.85$^{\pm0.02}$ & 0.83$^{\pm0.02}$ & 0.86$^{\pm0.00}$ \\ 
	raw       & 0.33 & 0.81$^{\pm0.01}$ & 0.80$^{\pm0.02}$ & 0.79$^{\pm0.02}$ & 0.79$^{\pm0.00}$ \\ 
\bottomrule
\end{tabular}

writing to /tmp/accuracy.tex



In [5]:
tableroot = "/tmp"
root = "../models/train/12classes"

tum_kappa, tum_kappa_std, gaf_kappa, gaf_kappa_std, tum_acc, tum_acc_std, gaf_acc, gaf_acc_std = get_acc_kappa(root)

rf_preprocessed_kappa = 0.71
rf_raw_kappa = 0.40
rf_preprocessed_acc = 0.75
rf_raw_acc = 0.47

print("kappa")
print()
table=make_table(gaf_kappa,gaf_kappa_std,tum_kappa,tum_kappa_std, "kappa", rf_pre=rf_preprocessed_kappa, rf_raw=rf_raw_kappa)
print(table)
print()
fname = os.path.join(tableroot,"kappa.tex")
print(f"writing to {fname}")
print(table,file=open(fname,'w'))
print()
print("accuracy")
print()
table=make_table(gaf_acc,gaf_acc_std,tum_acc,tum_acc_std, "accuracy", rf_pre=rf_preprocessed_acc, rf_raw=rf_raw_acc)
print(table)
print()
fname = os.path.join(tableroot,"accuracy.tex")
print(f"writing to {fname}")
print(table,
      file=open(fname,'w'))
print()

kappa

\begin{tabular}{lccccc}
\toprule
kappa & Random Forest & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}
    	processed & 0.71 & 0.87$^{\pm0.01}$ & 0.87$^{\pm0.03}$ & 0.85$^{\pm0.01}$ & 0.88$^{\pm0.02}$ \\ 
	raw       & 0.40 & 0.83$^{\pm0.00}$ & 0.82$^{\pm0.02}$ & 0.78$^{\pm0.02}$ & 0.71$^{\pm0.05}$ \\ 
\bottomrule
\end{tabular}

writing to /tmp/kappa.tex

accuracy

\begin{tabular}{lccccc}
\toprule
accuracy & Random Forest & \rnn & \transformer & \msresnet & \tempcnn \\
\cmidrule(lr){2-2}\cmidrule(lr){3-3}\cmidrule(lr){4-4}\cmidrule(lr){5-5}\cmidrule(lr){6-6}
    	processed & 0.75 & 0.92$^{\pm0.01}$ & 0.92$^{\pm0.02}$ & 0.91$^{\pm0.01}$ & 0.92$^{\pm0.01}$ \\ 
	raw       & 0.47 & 0.90$^{\pm0.00}$ & 0.89$^{\pm0.01}$ & 0.87$^{\pm0.01}$ & 0.83$^{\pm0.04}$ \\ 
\bottomrule
\end{tabular}

writing to /tmp/accuracy.tex

